In [3]:
import pandas as pd
import numpy as np
import xgboost as xgb
from tqdm import tqdm
from sklearn.svm import SVC
from keras.models import Sequential
# from keras.layers.recurrent import LSTM, GRU
# from keras.layers.core import Dense, Activation, Dropout
# from keras.layers.embeddings import Embedding
# from keras.layers.normalization.batch_normalization_v1 import BatchNormalization
# from keras.utils import np_utils
from sklearn import preprocessing, decomposition, model_selection, metrics, pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
# from keras.layers import GlobalMaxPooling1D, Conv1D, MaxPooling1D, Flatten, Bidirectional, SpatialDropout1D
# from keras.preprocessing import sequence, text
# from keras.callbacks import EarlyStopping
from nltk import word_tokenize
from nltk.corpus import stopwords
import sklearn.metrics as me
from sklearn.metrics import classification_report
import os
import pandas as pd
import numpy as np
from glob import glob
from ast import literal_eval
import jsonlines
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import plot_confusion_matrix
import random
from sklearn.utils import shuffle
seed = 2022

In [ ]:
#  count vectorizer, TFIDF, Doc2Vec, Universal Sentence Encoder (USE), and WordPiece.

In [17]:
train = pd.read_csv('/kaggle/input/opp-9-testandtrain/train/opp_bi_1st_Collection.csv')
test = pd.read_csv('/kaggle/input/opp-9-testandtrain/test/1st_Collection_test.csv')

In [18]:
train

,text,label
0,Privacy Policy Sci-News.com is committed to ...,non First Party Collection
1,Information that Sci-News.com May Collect Onli...,First Party Collection/Use
2,"- if you contact us, we may keep a record of t...",non First Party Collection
3,- details of your visits to our site including...,First Party Collection/Use
4,Sci-News.com does not knowingly collect or sol...,non First Party Collection
...,...,...
2610,Google Adsense and the DoubleClick DART Cookie...,non First Party Collection
2611,To opt out of the DART cookies you may visit t...,non First Party Collection
2612,Other Third Party ad servers or ad networks ma...,non First Party Collection
2613,Collection of Personal Information When visi...,First Party Collection/Use


In [19]:
lbl_enc = preprocessing.LabelEncoder()
y_test =  lbl_enc.fit_transform(test.label.values)
train = shuffle(train)
xtrain = train['text']
ytrain = lbl_enc.fit_transform(train.label.values)
x_test = test['text']

In [23]:
ytrain

array([1, 1, 1, ..., 0, 1, 1])

In [24]:
from nltk.stem import WordNetLemmatizer
# 初始化WordNetLemmatizer
wordnet_lemmatizer = WordNetLemmatizer()

#  wordslemmatization and stemming词性还原后的
xtrain_lemmatized = xtrain.apply(lambda x: ' '.join([wordnet_lemmatizer.lemmatize(word, pos="v") for word in x.split()]))
x_test_lemmatized = x_test.apply(lambda x: ' '.join([wordnet_lemmatizer.lemmatize(word, pos="v") for word in x.split()]))


In [25]:
xtrain

570      Children.  Chase Paymentech does not solicit ...
2531     Third-Party Relationships     We have several...
1565    If you are located outside the United States o...
1933    We have additional specific notices for our pe...
38      Security   This Web site takes every precautio...
                              ...                        
1239    We work to protect the security of your inform...
820                        Revision Date: March 24th 2015
292     In addition, Allstate and its business partner...
1499    Food Allergy Research & Education reserves the...
2304     Security of your Personal Information:    Abi...
Name: text, Length: 2615, dtype: object

In [26]:
xtrain_lemmatized

570     Children. Chase Paymentech do not solicit or k...
2531    Third-Party Relationships We have several type...
1565    If you be locate outside the United States of ...
1933    We have additional specific notice for our per...
38      Security This Web site take every precaution t...
                              ...                        
1239    We work to protect the security of your inform...
820                        Revision Date: March 24th 2015
292     In addition, Allstate and its business partner...
1499    Food Allergy Research & Education reserve the ...
2304    Security of your Personal Information: Abita b...
Name: text, Length: 2615, dtype: object

In [27]:
print (xtrain.shape)
print (x_test.shape)

(2615,)
(654,)


In [29]:
#tfidf，stopwords
tfv = TfidfVectorizer(min_df=3,  max_features=None, 
            strip_accents='unicode', analyzer='word',token_pattern=r'\w{1,}',
            ngram_range=(1, 3), use_idf=1,smooth_idf=1,sublinear_tf=1,
            stop_words = 'english')
tfv.fit(list(xtrain_lemmatized) + list(x_test_lemmatized))
xtrain_tfv =  tfv.transform(xtrain_lemmatized) 
x_test_tfv = tfv.transform(x_test_lemmatized)

In [30]:
#countVector
ctv = CountVectorizer(analyzer='word',token_pattern=r'\w{1,}',
            ngram_range=(1, 3), stop_words = 'english')

# Fitting Count Vectorizer to both training and test sets (semi-supervised learning)
ctv.fit(list(xtrain_lemmatized) + list(x_test_lemmatized))
xtrain_ctv =  ctv.transform(xtrain_lemmatized) 
x_test_ctv = ctv.transform(x_test_lemmatized)

In [57]:
#Doc2Vec
from sentence_transformers import SentenceTransformer, util
model = SentenceTransformer('all-distilroberta-v1')

xtrain_d2v = model.encode(xtrain_lemmatized)
x_test_d2v = model.encode(x_test_lemmatized)

Downloading:   0%|          | 0.00/737 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/190 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/10.3k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/653 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/116 [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/13.1k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/798k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/349 [00:00<?, ?B/s]

Batches:   0%|          | 0/82 [00:00<?, ?it/s]

Batches:   0%|          | 0/21 [00:00<?, ?it/s]

In [38]:
#use
import tensorflow as tf
import tensorflow_hub as hub
import numpy as np
module_url = "https://tfhub.dev/google/universal-sentence-encoder/4"
model = hub.load(module_url)

In [39]:
xtrain_use = model(xtrain_lemmatized)
x_test_use = model(x_test_lemmatized)

In [40]:
xtrain_use

<tf.Tensor: shape=(2615, 512), dtype=float32, numpy=
array([[-0.03332692,  0.01645822,  0.04689226, ...,  0.00547493,
        -0.05081613,  0.04121698],
       [-0.04335521, -0.06142476, -0.00312726, ...,  0.05457618,
        -0.04264261, -0.05761734],
       [-0.01068312,  0.06151609, -0.04308171, ...,  0.04409941,
         0.01368675, -0.04149782],
       ...,
       [ 0.00268837,  0.034666  , -0.03069793, ...,  0.00218617,
        -0.0313174 , -0.02673826],
       [ 0.05508587,  0.01639115,  0.0396737 , ...,  0.06999274,
        -0.03702337, -0.05722419],
       [-0.02825033,  0.0673234 , -0.00419519, ..., -0.02149228,
        -0.03531101, -0.04437394]], dtype=float32)>

In [42]:
def measure_func(yvalid, predictions, lbl_enc):
    decod_predictions = lbl_enc.inverse_transform(predictions)
    decod_yvalid = lbl_enc.inverse_transform(yvalid)
    report = classification_report(decod_yvalid, decod_predictions)
    print(report)
def confusion_matrix(yvalid, predictions, lbl_enc):
    decod_predictions = lbl_enc.inverse_transform(predictions)
    decod_yvalid = lbl_enc.inverse_transform(yvalid)
    matrix = me.confusion_matrix(yvalid, predictions)
    print(matrix)
    return matrix

In [ ]:
#lr cnn dnn svm rf

In [44]:
#lr
#tfidf classification 
clf = LogisticRegression(C=1.0)
clf.fit(xtrain_tfv, ytrain)
predictions = clf.predict(x_test_tfv)
measure_func(y_test, predictions, lbl_enc)

                            precision    recall  f1-score   support

First Party Collection/Use       0.94      0.63      0.76       260
non First Party Collection       0.80      0.97      0.88       394

                  accuracy                           0.84       654
                 macro avg       0.87      0.80      0.82       654
              weighted avg       0.86      0.84      0.83       654



In [46]:
confusion_matrix(y_test, predictions, lbl_enc)

[[165  95]
 [ 10 384]]


array([[165,  95],
       [ 10, 384]])

In [47]:
clf = LogisticRegression(C=1.0)
clf.fit(xtrain_ctv, ytrain)
predictions = clf.predict(x_test_ctv)
measure_func(y_test, predictions, lbl_enc)

                            precision    recall  f1-score   support

First Party Collection/Use       0.91      0.71      0.80       260
non First Party Collection       0.83      0.95      0.89       394

                  accuracy                           0.86       654
                 macro avg       0.87      0.83      0.84       654
              weighted avg       0.86      0.86      0.85       654



In [48]:
confusion_matrix(y_test, predictions, lbl_enc)

[[185  75]
 [ 19 375]]


array([[185,  75],
       [ 19, 375]])

In [59]:
clf = LogisticRegression(C=1.0)
clf.fit(xtrain_d2v, ytrain)
predictions = clf.predict(x_test_d2v)
measure_func(y_test, predictions, lbl_enc)

                            precision    recall  f1-score   support

First Party Collection/Use       0.39      0.05      0.09       260
non First Party Collection       0.60      0.95      0.74       394

                  accuracy                           0.59       654
                 macro avg       0.50      0.50      0.41       654
              weighted avg       0.52      0.59      0.48       654



In [64]:
# from umap import UMAP 
# from sklearn.preprocessing import MinMaxScaler # Scale features to [0,1] range 
# X_scaled = MinMaxScaler().fit_transform(xtrain_d2v)
# # Initialize and fit UMAP 
# mapper = UMAP(n_components=2, metric="cosine").fit(X_scaled) 
# # Create a DataFrame of 2D embeddings 
# df_emb = pd.DataFrame(mapper.embedding_, columns=["X", "Y"]) 
# df_emb["label"] = ytrain 
# df_emb.head()
# fig, axes = plt.subplots(2, 3, figsize=(7,5)) 
# axes = axes.flatten() 
# cmaps = ["Greys", "Blues"] 
# labels = ytrain 
# for i, (label, cmap) in enumerate(zip(labels, cmaps)): 
#     df_emb_sub = df_emb.query(f"label == {i}") 
#     axes[i].hexbin(df_emb_sub["X"], df_emb_sub["Y"], cmap=cmap, gridsize=20, linewidths=(0,)) 
#     axes[i].set_title(label) 
#     axes[i].set_xticks([]), axes[i].set_yticks([]) 
# plt.tight_layout() 
# plt.show()


,X,Y,label
0,1.293881,6.771125,1
1,1.145383,6.707828,1
2,-0.943426,5.150518,1
3,-4.007936,3.122122,1
4,4.883119,3.248899,1


In [62]:
confusion_matrix(y_test, predictions, lbl_enc)

[[  1 259]
 [  0 394]]


array([[  1, 259],
       [  0, 394]])

In [50]:
confusion_matrix(y_test, predictions, lbl_enc)

[[  9 251]
 [ 21 373]]


array([[  9, 251],
       [ 21, 373]])

In [51]:
clf = LogisticRegression(C=1.0)
clf.fit(xtrain_use, ytrain)
predictions = clf.predict(x_test_use)
measure_func(y_test, predictions, lbl_enc)

                            precision    recall  f1-score   support

First Party Collection/Use       0.89      0.72      0.80       260
non First Party Collection       0.84      0.94      0.89       394

                  accuracy                           0.85       654
                 macro avg       0.86      0.83      0.84       654
              weighted avg       0.86      0.85      0.85       654



In [83]:
from gensim.test.utils import common_texts
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
documents = [TaggedDocument(doc, [i]) for i, doc in enumerate(list(xtrain_lemmatized) + list(x_test_lemmatized))]


In [84]:
model = Doc2Vec(documents, vector_size=200, window=4, min_count=1, workers=4)

In [96]:
xtrain_lemmatized.head()

570     Children. Chase Paymentech do not solicit or k...
2531    Third-Party Relationships We have several type...
1565    If you be locate outside the United States of ...
1933    We have additional specific notice for our per...
38      Security This Web site take every precaution t...
Name: text, dtype: object

In [85]:
xtrain_d2v = model.infer_vector(xtrain_lemmatized)
x_test_d2v = model.infer_vector(x_test_lemmatized)

In [97]:
xtrain_d2v = []
for row in xtrain_lemmatized:
    vec = model.infer_vector([row])
    xtrain_d2v.append(vec)

In [98]:
x_test_d2v = []
for row in x_test_lemmatized:
    vec = model.infer_vector([row])
    x_test_d2v.append(vec)

In [103]:
from sklearn.metrics import accracy_score
clf = LogisticRegression(C=1e10, classweight)
clf.fit(xtrain_d2v, ytrain)
predictions = clf.predict(x_test_d2v)
measure_func(y_test, predictions, lbl_enc)

                            precision    recall  f1-score   support

First Party Collection/Use       0.38      0.15      0.22       260
non First Party Collection       0.60      0.84      0.70       394

                  accuracy                           0.56       654
                 macro avg       0.49      0.49      0.46       654
              weighted avg       0.51      0.56      0.51       654



In [105]:
from sklearn.metrics import accuracy_score

In [106]:
accuracy_score(y_test,predictions)

0.5642201834862385